In [1]:
%load_ext autoreload
%autoreload 2
import vtspy

In [2]:
import glob
from pathlib import Path
import re
import os
from IPython.display import clear_output
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from backend import create_app
from backend import utils
from flask_sqlalchemy import SQLAlchemy as sal

app = create_app()
app.app_context().push()

In [4]:
import flask_sqlalchemy

In [5]:
os.system("rm backend/database/*")

from backend.models import db
db.init_app(app)

from backend.models import RunByRun, Sources

In [6]:
def empty_image(base_dir):
    plt.figure(figsize=(8,7))
    plt.xticks([])
    plt.yticks([])
    plt.savefig(f"{base_dir}/veritas/fit.png")
    plt.figure(figsize=(6,4))
    plt.xticks([])
    plt.yticks([])
    plt.savefig(f"{base_dir}/veritas/sed.png")

In [7]:
os.system("rm backend/database/*")

from backend.models import db
db.init_app(app)
db.create_all()

fit_files = glob.glob("./backend/data/*")
from backend.models import RunByRun, Sources


list_from_flx = ["e_min", "e_max", "e2dnde", "e2dnde_err", 
                 "e2dnde_errp", "e2dnde_errn", "e2dnde_ul", 
                 "ts", "is_ul", "stat"] 

src_id = 1
for fi, file in enumerate(fit_files):

    file_path = Path(file)
    run_num = re.findall("([0-9]+)", file_path.name)[0]
    base_dir = f"./backend/data/{run_num}/"
    
    if not(os.path.exists(base_dir)):
        os.system(f"mkdir {base_dir}")
    config_file = base_dir+"config.yaml"
    
    file = f"{base_dir}/veritas/{run_num}.anasum.fits"
    vtspy.JointConfig(file, config_file=config_file, 
                      fermi_outdir = base_dir+"/fermi/", 
                      fermi_data = base_dir+"/fermi/",
                      joint_outdir = base_dir+"/joint/",
                      veritas_outdir = base_dir+"/veritas/", verbosity=0)
    veritas = vtspy.VeritasAnalysis(config_file=config_file, overwrite=True, verbosity=0, simbad=False)
    veritas.construct_dataset(eff_cut=10, bias_cut=10)
    empty_image(base_dir)
    output = {}
    veritas.fit(model="PowerLaw")
    veritas.peek_dataset(save_fig=True, filename=f"{base_dir}/veritas/dataset.png")
    fit_result = veritas.fit_results.success
    if fit_result:
        fit_result = veritas.fit_results.models.parameters.to_dict()
        for d in fit_result:
            output["pl_"+d["name"]] = d["value"]
            output["pl_"+d["name"]+"_err"] = d["error"]
        
        if (output["pl_amplitude"] < 0) or (output["pl_index_err"]>3):
            output["pl_fit_flag"] = "C"
        else:
            try:
                veritas.analysis()
                veritas.plot("sed", save_fig=True, filename=f"{base_dir}/veritas/sed.png")
            except:
                pass
            
            veritas.analysis(energy_bins = [100*u.GeV, 30*u.TeV])
            tab = veritas.print_flux()
            
            for par in list_from_flx:
                output["pl_"+par] = float(tab[par][0])
            if output["pl_is_ul"]:
                if output["pl_e2dnde_ul"] > 0:
                    output["pl_fit_flag"] = "B"
                else:
                    output["pl_fit_flag"] = "C"
            else:
                if output["pl_e2dnde"] > 0:
                    output["pl_fit_flag"] = "A"
                else:
                    output["pl_fit_flag"] = "C"
            
        try:
            veritas.plot("fit", save_fig=True, filename=f"{base_dir}/veritas/fit.png")
        except:
            pass
        
    else:
        output["pl_fit_flag"] = "D"
#     veritas.fit(model="LogParabola")
#     output["log_fit"] = veritas.fit_results.success
#     if output["log_fit"]:
#         fit_result = veritas.fit_results.models.parameters.to_dict()
#         for d in fit_result:
#             output["log_"+d["name"]] = d["value"]
#             output["log_"+d["name"]+"_err"] = d["error"]
#         veritas.analysis(energy_bins = [100*u.GeV, 30*u.TeV])
#         tab = veritas.print_flux()   
#         for par in list_from_flx:
#             output["log_"+par] = float(tab[par][0])

    deadtime = (veritas.info_table["livetime"][0]-veritas.info_table["ontime"][0])/veritas.info_table["livetime"][0]
    srcl = Sources.query.filter((Sources.name.ilike(veritas.target_name))).all()
    veritas.load_VDB()
    output["weather_flag"] = veritas.vdb_table["Weather"][0]
    output["run_flag"] = veritas.vdb_table["Status"][0]
    output["wobble"] = veritas.vdb_table["wobble"][0]
    
    if len(srcl) == 0:
        src = Sources(src_id,
                      veritas.target_name,
                      veritas.config["selection"]["ra"],
                      veritas.config["selection"]["dec"],
                      int(veritas._N_on),
                      int(veritas._N_off),
                      float(veritas._alpha),
                      np.nan_to_num(float(veritas._sigma)),
                      float(veritas.info_table["ontime"][0]),
                     )
        db.session.add(src)
        db.session.commit()
        src_id +=1
    else:
        src = srcl[0]
        src.N_on += int(veritas._N_on)
        src.N_off += int(veritas._N_off)
        src.alpha = (src.alpha*src.exposure+np.nan_to_num(float(veritas._alpha))*float(veritas.info_table["ontime"][0]))/(src.exposure+float(veritas.info_table["ontime"][0]))
        src.exposure += float(veritas.info_table["ontime"][0])
        src.sigma = np.nan_to_num(utils.LiMaSiginficance(src.N_on, src.N_off, src.alpha))
        db.session.commit()
        
    rbr = RunByRun(veritas.info_table["name"][0], 
             Sources.query.filter_by(name = veritas.target_name).all()[0].src_id,
             veritas.config["selection"]["tmin"],
             veritas.config["selection"]["tmax"],
             int(veritas._N_on),
             int(veritas._N_off),
             float(veritas.info_table["excess"][0]),
             float(veritas._alpha),
             float(veritas.info_table["ontime"][0]),
             float(veritas.info_table["livetime"][0]),
             float(deadtime),
             float(veritas._sigma))
    
    
    for o in output:
        setattr(rbr, o, output[o])
    
    db.session.add(rbr)
    db.session.commit()
    clear_output() 

